In [1]:
# # 1. Import thư viện cần thiết
# from google.colab import drive
# drive.mount('/content/drive')


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 2. Đọc file csv và gắng index với giá Close
df = pd.read_csv('F:/hk2 nam3/nhi/is403/IS403_O21_04/Dataset/DIG Historical Data 2018.csv')
df['Price'] = df['Price'].str.replace('$', '', regex=True)
df['Price'] = df['Price'].str.replace(',', '', regex=True)
df['Price'] = df.Price.astype(float)
df = df[::-1]
df1=df.reset_index()['Price']
df1

0       11033.2
1       11457.6
2       12094.1
3       12094.1
4       12200.2
         ...   
1547    30450.0
1548    29900.0
1549    30400.0
1550    31700.0
1551    31950.0
Name: Price, Length: 1552, dtype: float64

In [4]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [5]:
# 4. Chia train test
train_size = int(0.8 * len(df1))
test_size = int(0.2 * len(df1))

train_data = df1[:train_size]
test_data = df1[train_size:]

In [6]:
test_size

310

In [7]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

 #6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [8]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [9]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [10]:
X_train.shape

(1140, 100, 1)

In [11]:
from keras.layers import LSTM, Dense, Activation

def swish(x):
    return x * tf.keras.backend.sigmoid(x)

model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(100, 1)))
model.add(Activation(swish))
model.add(LSTM(32, return_sequences=True))
model.add(Activation(swish))
model.add(LSTM(32))
model.add(Activation(swish))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')



In [12]:
# 9. Fit mô hình với dữ liệu train
# model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for improvement
    patience=3,         # Stop training after 3 epochs of no improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

model.fit(X_train, y_train,
          validation_data=(X_test, ytest),
          epochs=100,
          batch_size=64,
          verbose=1,
          callbacks=[early_stopping])

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 36s 349ms/step - loss: 0.0602 - val_loss: 0.0018
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - loss: 0.0111 - val_loss: 0.0067
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 147ms/step - loss: 0.0059 - val_loss: 0.0015
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step - loss: 0.0047 - val_loss: 6.2228e-04
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - loss: 0.0035 - val_loss: 5.6633e-04
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - loss: 0.0041 - val_loss: 0.0010
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 205ms/step - loss: 0.0033 - val_loss: 0.0011
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 171ms/step - loss: 0.0035 - val_loss: 6.0063e-04


In [13]:
# 10. Dự báo dữ liệu test, val
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)

36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 108ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [14]:
# 11. Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)

In [15]:
y_pred.shape

(210, 1)

In [16]:
# 12. Đánh giá độ chính xác thuật toán bằng RMSE
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
test_mape = np.mean(np.abs((ytest - y_pred) / ytest)) * 100
test_mae = np.mean(np.abs(ytest - y_pred))


print('Testing RMSE:', test_rmse)
print('Testing RMSE:', test_rmse)
print('Testing MAE:', test_mae)

Testing RMSE: 25217.77890268271
Testing RMSE: 25217.77890268271
Testing MAE: 25031.449088483285


In [ ]:
# 13. Dự báo 30 ngày tiếp theo
x_input=test_data[60:].reshape(1,-1)
x_input.shape

temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=250
i=0
while(i<30):

    if(len(temp_input)>250):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


0 day input [0.06722566 0.07268388 0.06886313 0.07159224 0.06886313 0.075413
 0.0852378  0.09069602 0.09724589 0.10161247 0.11362056 0.11798714
 0.11907878 0.11580385 0.11798714 0.1048874  0.10597905 0.11798714
 0.11362056 0.11525803 0.1147122  0.11252891 0.1048874  0.11252891
 0.11907878 0.13217852 0.13436181 0.13599928 0.13381599 0.12999523
 0.13709092 0.1512823  0.15346559 0.14855319 0.15728635 0.15510306
 0.13927421 0.15182812 0.15182812 0.15291977 0.15564888 0.15237395
 0.15510306 0.1561947  0.15674052 0.1611071  0.16984026 0.1660195
 0.15946964 0.1611071  0.16820279 0.15510306 0.1561947  0.14909901
 0.15674052 0.15182812 0.15237395 0.15510306 0.15946964 0.16656533
 0.16492786 0.17966506 0.17693595 0.18021088 0.17748177 0.17256937
 0.15837799 0.15783217 0.1561947  0.16274457 0.16001546 0.15564888
 0.15510306 0.16165292 0.15837799 0.15783217 0.16929444 0.17256937
 0.18075671 0.18294    0.17857342 0.18130253 0.19876884 0.20095213
 0.207502   0.20586453 0.21186858 0.207502   0.211868

In [ ]:
# 14.Vẽ hình

train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size+1, step=1)

plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size+1, step=1)
plt.plot(test_data_index,(y_pred))
prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Predict','Predict30days'])
plt.show()